In [ ]:
import time
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException, WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import re

# Se inicializa el data frame y la URL
all_info = pd.DataFrame()
url = 'https://capr.org/account/#app/member-directory/?name=&last=&license=&especialidad=&delegacion=&searching=true'
#url = 'https://capr.org/account/#app/member-directory/?name=&last=&license=&especialidad=&delegacion=&searching=true&page=1'
#url = 'https://capr.org/account/#app/member-directory/?name=&last=&license=&especialidad=&delegacion=&searching=true&page=10'

# Función que se va a aplicar a la info obtenida del scrap

In [365]:
def parse_abogado_info(texto: str) -> dict:
    lines = [line.strip() for line in texto.strip().split("\n") if line.strip()]
    data = {}

    field_map = {
        "Número de Colegiación:": "colegiacion",
        "RUA:": "rua",
        "Correo Electrónico": "correo",
        "Tel. Residencial": "tel_residencial",
        "Tel. Oficina": "tel_oficina",
        "Tel. Celular": "tel_celular",
        "Otro": "otro",
        "Especialidades": "especialidades",
        "Delegación:": "delegacion",
    }

    # Nombre siempre es la primera línea
    data["nombre"] = lines[0] if lines else ""
    i = 1

    while i < len(lines):
        line = lines[i]
        matched = False

        for label, key in field_map.items():
            if label.endswith(":") and line.startswith(label):
                data[key] = line.split(label)[1].strip()
                matched = True
                break

            elif line == label:
                if i + 1 < len(lines) and lines[i + 1] not in field_map:
                    data[key] = lines[i + 1].strip()
                    i += 1
                else:
                    data[key] = ""
                matched = True
                break

        i += 1

    return data

In [410]:
def parse_abogado_info_2(texto: str) -> dict:
    lines = [line.strip() for line in texto.strip().split("\n") if line.strip()]
    data = {}

    field_map = {
        "Número de Colegiación:": "colegiacion",
        "RUA:": "rua",
        "Correo Electrónico": "correo",
        "Tel. Residencial": "tel_residencial",
        "Tel. Oficina": "tel_oficina",
        "Tel. Celular": "tel_celular",
        "Otro": "otro",
        "Especialidades": "especialidades",
        "Delegación:": "delegacion",
    }

    # Extraer lista normalizada de claves para comparación rápida
    all_labels = list(field_map.keys())
    

    data["nombre"] = lines[0] if lines else ""
    i = 1

    while i < len(lines):
        line = lines[i].strip()
        #print(line)
        matched = False

        for label, key in field_map.items():
            # Caso 1: Etiqueta con ":" y valor en la misma línea
            if label.endswith(":") and line.startswith(label):
                value = line[len(label):].strip()
                data[key] = value
                matched = True
                break

            elif line == label:
                next_line = lines[i + 1].strip() if i + 1 < len(lines) else ""
            
                # Verifica si next_line es una nueva etiqueta
                es_label = (
                    next_line in all_labels or
                    any(next_line.startswith(lbl) for lbl in all_labels if lbl.endswith(":"))
                )
            
                if es_label:
                    data[key] = ""
                else:
                    data[key] = next_line
                    i += 1  # Consumir la línea del valor
            
                matched = True
                break

        i += 1

    return data

## Scrapper para una sola pagina (sin for)

In [ ]:
# Configura el driver
options = Options()
driver = webdriver.Firefox(options=options)

# Try except para manejar la carga de la página y el acceso al shadow DOM
try:
    driver.set_page_load_timeout(15)
    driver.get(url)
    # Esperar explícitamente a que capr-app esté presente en el DOM
    #WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div')))

    # Espera a que el shadow host (tag que tiene al shadow-roor dentro) esté presente
    shadow_host = driver.find_element(By.XPATH, '//*[@id="content"]/div')

    # Ejecuta JS para acceder al shadow root
    shadow_root = driver.execute_script("return arguments[0].shadowRoot", shadow_host)
    # Esperar a que un div con clase 'row' aparezca dentro del shadow DOM
    WebDriverWait(driver, 15).until(
        lambda d: d.execute_script("""
            const shadowHost = arguments[0];
            if (!shadowHost || !shadowHost.shadowRoot) return false;

            return shadowHost.shadowRoot.querySelector('div.row') !== null;
        """, shadow_host)
    )
    
    # Ahora puedes buscar dentro del shadow root
    rows_single = shadow_root.find_elements(By.CSS_SELECTOR, 'div.row') 
    #Extraer el texto del segundo elemento de la lista
except TimeoutException:
    print("Loading took too much time!")
    driver.quit()
except Exception as e:
    print(f"Failed to load page due to error: {e}")
    driver.quit()


In [333]:
#Extraer los datos del row antes de cerrar el driver
for i in range(1, len(rows_single),2):
    data = rows_single[i].text
    print(parse_abogado_info_2(data))

{'nombre': 'ACEVEDO CRUZ, GLORIMAR', 'colegiacion': '13371', 'rua': '12146', 'correo': '', 'tel_residencial': '', 'tel_oficina': '', 'tel_celular': '', 'otro': '', 'especialidades': 'Criminal,Familia,Laboral', 'delegacion': 'Delegación de Río Piedras'}
{'nombre': 'ACEVEDO CRUZ, JUAN R', 'colegiacion': '6223', 'rua': '5062', 'correo': 'jr@jracevedo.com', 'tel_residencial': '', 'tel_oficina': '787-751-2341', 'tel_celular': '787-317-2575', 'otro': '', 'especialidades': 'Criminal', 'delegacion': 'Delegación de Bayamón'}
{'nombre': 'ACEVEDO GARCIA, JASHIRA', 'colegiacion': '21544', 'rua': '23581', 'correo': '', 'tel_residencial': '', 'tel_oficina': '', 'tel_celular': '', 'otro': '', 'especialidades': '', 'delegacion': 'Delegación de Ponce'}
{'nombre': 'ACEVEDO LOPEZ, LUTGARDO', 'colegiacion': '20636', 'rua': '21664', 'correo': 'lacevedolaw@gmail.com', 'tel_residencial': '', 'tel_oficina': '', 'tel_celular': '787-466-9845', 'otro': '', 'especialidades': 'Administrativo,Civil,Contratos,Contri

In [329]:
parse_abogado_info_2(rows_single[5].text)

({'nombre': 'ACEVEDO GARCIA, JASHIRA',
  'colegiacion': '21544',
  'rua': '23581',
  'correo': '',
  'tel_residencial': '',
  'tel_oficina': '',
  'tel_celular': '',
  'otro': '',
  'especialidades': '',
  'delegacion': 'Delegación de Ponce'},
 ['Número de Colegiación:',
  'RUA:',
  'Correo Electrónico',
  'Tel. Residencial',
  'Tel. Oficina',
  'Tel. Celular',
  'Otro',
  'Especialidades',
  'Delegación:'])

## Test con funciones separadas

### Función obtener rows para validar el DOM

In [380]:
def obtener_rows(driver, url):
    """
    Devuelve los elementos <div class="row"> desde el Shadow DOM.
    Reintenta si el DOM cambia y causa errores 'stale'.
    """
    for intento in range(5):
        try:
            driver.get(url)
            # Buscar shadow host
            shadow_host = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div'))
            )

            # Refrescar shadow_root
            shadow_root = driver.execute_script("return arguments[0].shadowRoot", shadow_host)

            # Verificar que <div class="row"> esté disponible
            WebDriverWait(driver, 10).until(
                lambda d: d.execute_script("""
                    const root = arguments[0].shadowRoot;
                    return root && root.querySelectorAll('div.row').length > 0;
                """, shadow_host)
            )

            # Obtener los rows (si el DOM es válido)
            rows = driver.execute_script("""
                return Array.from(arguments[0].shadowRoot.querySelectorAll('div.row'));
            """, shadow_host)

            return rows
        except StaleElementReferenceException:
            print(f"Intento {intento + 1}: DOM cambió, reintentando...")
            time.sleep(1)
        except Exception as e:
            print(f"Error inesperado al obtener Shadow DOM: {e}")
            return []

    raise Exception("No se pudo acceder a los rows del Shadow DOM tras múltiples intentos.")

In [ ]:
# Scraper
base_url = "https://capr.org/account/#app/member-directory/?name=&last=&license=&especialidad=&delegacion=&searching=true"
list_abogados = []

# Configura el driver
options = Options()
driver = webdriver.Firefox(options=options)

try:
    for i in range(3):
        url_actual = base_url if i == 0 else f"{base_url}&page={i}"
        print(f"Iteración {i+1}, accediendo a: {url_actual}")
        try:
            driver.set_page_load_timeout(15)
            rows = obtener_rows(driver, url_actual)
            for j in range(1, len(rows), 2):
                text_data = rows[j].text
                parsed = parse_abogado_info_2(text_data)
                list_abogados.append(parsed)
        except TimeoutException:
            print("Timeout al cargar la página")
            continue
        except WebDriverException as e:
            print(f"Reiniciando driver por error: {e}")
            driver.quit()
            driver = webdriver.Firefox(options=options)
            continue
finally:
    driver.quit()

Iteración 1, accediendo a: https://capr.org/account/#app/member-directory/?name=&last=&license=&especialidad=&delegacion=&searching=true
Iteración 2, accediendo a: https://capr.org/account/#app/member-directory/?name=&last=&license=&especialidad=&delegacion=&searching=true&page=1
Iteración 3, accediendo a: https://capr.org/account/#app/member-directory/?name=&last=&license=&especialidad=&delegacion=&searching=true&page=2


## For del Try/Except para hacer scrap de todas las paginas

In [ ]:
# CREACIÓN DEL DRIVER
options = Options()
driver = webdriver.Firefox(options=options)
# Conteo
cont = 0
list_datos = []
#for i in range(0, 272):
for i in range(0, 30):
    if i == 0:
        url_actual = url
        print(f"accediendo a: {url_actual}")
    else:
        url_actual = url + f"&page={cont}"
        
    try:
        print(f"iteracion {i}, accediendo a: {url_actual}")
        driver.set_page_load_timeout(15)
        driver.get(url_actual)
        # Esperar explícitamente a que capr-app esté presente en el DOM
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div')))
        WebDriverWait(driver, 10).until(lambda d: d.find_element(By.XPATH, '//*[@id="content"]/div'))

        for _ in range(3):  # intentar 3 veces
            try:
                shadow_host = driver.find_element(By.XPATH, '//*[@id="content"]/div')
                shadow_root = driver.execute_script("return arguments[0].shadowRoot", shadow_host)
                break
            except StaleElementReferenceException:
                time.sleep(1)  # espera y vuelve a intentar
        # Espera a que el shadow host (tag que tiene al shadow-roor dentro) esté presente
        #shadow_host = driver.find_element(By.XPATH, '//*[@id="content"]/div')
        #shadow_root = driver.execute_script("return arguments[0].shadowRoot", shadow_host)
        # Esperar a que un div con clase 'row' aparezca dentro del shadow DOM
        WebDriverWait(driver, 15).until(
            lambda d: d.execute_script("""
                const shadowHost = arguments[0];
                if (!shadowHost || !shadowHost.shadowRoot) return false;
                return shadowHost.shadowRoot.querySelector('div.row') !== null;
            """, shadow_host)
        )
        # Ahora puedes buscar dentro del shadow root
        rows = shadow_root.find_elements(By.CSS_SELECTOR, 'div.row') 
        
        #Extraer los datos del row antes de cerrar el driver
        for i in range(1, len(rows),2):
            data = rows[i].text
            #print(parse_abogado_info_2(data))
            list_datos.append(parse_abogado_info_2(data))
        
    except TimeoutException:
        print("Loading took too much time!")
        driver.quit()
    except Exception as e:
        print(f"Failed to load page due to error: {e}")
        driver.quit()
    cont += 1


In [421]:
# Configurar WebDriver
options = Options()
options.add_argument("--headless")
driver = webdriver.Firefox(options=options)

# Variables de control
cont = 0
list_datos = []
name_inicial = "A"  # Nombre inicial para filtrar
inicio = time.time()
# Loop de páginas
for i in range(1, 272):
    url_actual = url if i == 0 else url + f"&page={cont}"
    print(f"\nIteración {i}, accediendo a: {url_actual}")
    try:
        driver.set_page_load_timeout(20)
        driver.get(url_actual)

        # Esperar que cargue shadow host
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div')))

        # Intentar acceder al shadow DOM con reintentos
        for attempt in range(3):
            try:
                shadow_host = driver.find_element(By.XPATH, '//*[@id="content"]/div')
                # Confirmar que hay contenido útil
                WebDriverWait(driver, 15).until(
                lambda d: (
                    d.execute_script("""
                        const host = document.querySelector('#content > div');
                        if (!host || !host.shadowRoot) return null;
                        const rows = host.shadowRoot.querySelectorAll('div.row');
                        return rows.length > 1 ? rows[1].innerText.split('\\n')[0].trim() : null;
                    """) != name_inicial
                ))
                
                WebDriverWait(driver, 10).until(
                    lambda d: d.execute_script("""
                        const el = arguments[0].shadowRoot.querySelector('div.row');
                        return el !== null;
                    """, shadow_host)
                )
                shadow_root = driver.execute_script("return arguments[0].shadowRoot", shadow_host)
                break  # si todo fue exitoso, salimos del retry loop
            except StaleElementReferenceException:
                print("Elemento stale, reintentando...")
                time.sleep(1)
                continue
        else:
            print("No se pudo acceder al shadow DOM.")
            continue  # saltar a la siguiente página

        # Extraer datos
        rows = shadow_root.find_elements(By.CSS_SELECTOR, 'div.row')
        #corroborar que hay datos y el primer nombre
        if len(rows) > 1:
            name_inicial = rows[1].text.split("\n")[0].strip()
            print(f"Nombre actual: {name_inicial}")
        else:
            print("No hay datos en esta página.")
            continue

        # Almacenar los datos en la lista
        for j in range(1, len(rows), 2):
            data = rows[j].text
            list_datos.append(parse_abogado_info_2(data))

    except TimeoutException:
        print("⏰ Timeout al cargar la página.")
        continue  # No cerrar el driver, solo saltar página
    except WebDriverException as e:
        print(f"WebDriverException: {e}")
        break  # cortar la ejecución
    except Exception as e:
        print(f"Error inesperado: {e}")
        continue

    cont += 1
    actual_time = time.time() - inicio
    #print(f"Tiempo actual: {actual_time:.2f} segundos")

# Cerrar el driver fuera del loop
print(f"tiempo total: {time.time() - inicio:.2f} segundos")
driver.quit()


Iteración 1, accediendo a: https://capr.org/account/#app/member-directory/?name=&last=&license=&especialidad=&delegacion=&searching=true&page=0
Nombre actual: ABENDAÑO EZQUERRO, JAVIER

Iteración 2, accediendo a: https://capr.org/account/#app/member-directory/?name=&last=&license=&especialidad=&delegacion=&searching=true&page=1
Nombre actual: ACEVEDO CRUZ, GLORIMAR

Iteración 3, accediendo a: https://capr.org/account/#app/member-directory/?name=&last=&license=&especialidad=&delegacion=&searching=true&page=2
Nombre actual: ACOSTA CABAN, ALEXANDRA MARINA

Iteración 4, accediendo a: https://capr.org/account/#app/member-directory/?name=&last=&license=&especialidad=&delegacion=&searching=true&page=3
Nombre actual: ADORNO GONZALEZ, NIURKA I

Iteración 5, accediendo a: https://capr.org/account/#app/member-directory/?name=&last=&license=&especialidad=&delegacion=&searching=true&page=4
Nombre actual: AGRAIT LLADO, BLANCA E

Iteración 6, accediendo a: https://capr.org/account/#app/member-direct

In [422]:
#df =pd.DataFrame(list_abogados)
df = pd.DataFrame(list_datos)
df.tail()

,nombre,colegiacion,rua,correo,tel_residencial,tel_oficina,tel_celular,otro,especialidades,delegacion
2695,"VILLANUEVA RIVERA, EDGAR",20646,21128,,,787-908-7059,,,"Apelativa,Civil,Constitucional,Contratos,Crimi...",Delegación de Bayamón
2696,"VILLANUEVA RODRIGUEZ, ALFREDO",8629,7341,,,,,,Corporativo,Delegación de Bayamón
2697,"VILLANUEVA SANCHEZ, MELISSA M",21173,22877,,,,,,,Delegación de Arecibo
2698,"VILLARES SEÑERIZ, MARTA M",12691,11550,,,,,,,Delegación de Caguas
2699,"VILLARINI BAQUERO, DANIEL",13636,12365,,,,,,,Delegación de San Juan


In [426]:
df.to_excel("datos_abogados.xlsx", index=False)

# Función para limpiar los datos de la tabla

In [245]:

list_datos = []
for i in range(1, len(rows),2):
    data = rows[i].text
    #print(parse_abogado_info_2(data))
    list_datos.append(parse_abogado_info(data))

#print(list_datos[0:5])  # Muestra solo el segundo elemento de la lista

df = pd.DataFrame(list_datos)
df.head(5)  # Muestra las primeras 5 filas del DataFrame

#print(f"Total de abogados encontrados: {list_datos}")

,nombre,colegiacion,rua,correo,tel_residencial,tel_oficina,tel_celular,otro,especialidades,delegacion
0,"ACOSTA CABAN, ALEXANDRA MARINA",20687,18993,,,,,,Delegación: Delegación de Bayamón,NaN
1,"ACOSTA FEBO, MELBA I",12600,11309,acostamelba@mac.com,,787-759-9292,,,"Contratos,Contributivo,Corporativo,Hipotecario...",Delegación de Río Piedras
2,"ACOSTA FEBO, ZULMA",13086,11817,,,,,,Delegación: Delegación de Caguas,NaN
3,"ACOSTA GONZALEZ, BENJAMIN",5918,4584,benacosta@lobajr.com,,787-722-2363,,,"Civil,Contratos,Daños y perjuicios,Federal,Lit...",Delegación de San Juan
4,"ACOSTA MORALES, KARLA J",19699,19502,lcda.acostamorales@gmail.com,,,,,Delegación: Delegación de Bayamón,NaN


In [27]:
all_info['info_url'] = [item.get_attribute('href') for item in driver.find_elements(By.XPATH, info_items["inf_url"])]
all_info['name'] = [item.text.strip() for item in driver.find_elements(By.XPATH, info_items["names"])]
all_info['title'] = [item.text.strip() for item in driver.find_elements(By.XPATH, info_items["titles"])]
all_info['company'] = [item.text.strip() for item in driver.find_elements(By.XPATH, info_items["companies"])]
all_info

,info_url,name,title,company
0,https://app.swapcard.com/event/offshorealert-l...,Team #1 Conflict International Limited,Investigator,Conflict International Limited
1,https://app.swapcard.com/event/offshorealert-l...,Farooq Ahmad Mann,Managing Director,Mann & Associates PAC
2,https://app.swapcard.com/event/offshorealert-l...,Jaafar Al-Fekaiki,Investigator,Freelancer
3,https://app.swapcard.com/event/offshorealert-l...,Prasanna Amarasinghe,Manager,Grant Thornton UK LLP
4,https://app.swapcard.com/event/offshorealert-l...,Jacob Atkins,Senior Reporter,Global Trade Review
...,...,...,...,...
210,https://app.swapcard.com/event/offshorealert-l...,Abby Wilson,Investigative Journalism Student,"City, University of London"
211,https://app.swapcard.com/event/offshorealert-l...,Colin Wilson,Director,KROLL
212,https://app.swapcard.com/event/offshorealert-l...,Dan Wise,Partner,Martin Kenney & Co (MKS)
213,https://app.swapcard.com/event/offshorealert-l...,Iryna Wiseman,Director,Quantuma Advisory Limited


In [72]:
for index, row in all_info.iloc[47:].iterrows():
    print(index)
    try:
        driver.get(all_info.loc[index,'info_url'])
        # Wait for the specific element to be visible
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "(//button[@class='sc-5a4b0056-0 bBWtJG'])[1]")))
        #soup = BeautifulSoup(driver.page_source, 'html.parser')
        #all_info.loc[index,'interest'] =  "" if driver.find_elements(By.XPATH, info_items['interest']) == [] else driver.find_element(By.XPATH, info_items['interest']).text.replace('\n'," ,")
        #all_info.loc[index,'profession'] =  "" if driver.find_elements(By.XPATH, info_items['profession']) == [] else driver.find_element(By.XPATH, info_items['profession']).text
        all_info.loc[index,'location'] = "" if driver.find_elements(By.XPATH, info_items['location']) == [] else driver.find_element(By.XPATH, info_items['location']).text
    except TimeoutException:
        print("Loading took too much time!")
    except Exception as e:
        print(f"Failed to load page due to error: {e} in index {index}")
        raise
    time.sleep(2)

47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
Loading took too much time!
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214


In [73]:
all_info

,info_url,name,title,company,interest,profession,location
0,https://app.swapcard.com/event/offshorealert-l...,Team #1 Conflict International Limited,Investigator,Conflict International Limited,"Asset Recovery ,Compliance ,Corruption ,Crypto...",Investigator,United Kingdom (UK)
1,https://app.swapcard.com/event/offshorealert-l...,Farooq Ahmad Mann,Managing Director,Mann & Associates PAC,"Asset Recovery ,Compliance ,Crypto ,Fraud ,Int...",Insolvency Practitioner,Singapore
2,https://app.swapcard.com/event/offshorealert-l...,Jaafar Al-Fekaiki,Investigator,Freelancer,"Asset Recovery ,Compliance ,Corruption ,Intell...",Investigator,United Kingdom (UK)
3,https://app.swapcard.com/event/offshorealert-l...,Prasanna Amarasinghe,Manager,Grant Thornton UK LLP,"Asset Recovery ,Corruption ,Crypto ,Fraud ,Inv...",Investigator,United Kingdom (UK)
4,https://app.swapcard.com/event/offshorealert-l...,Jacob Atkins,Senior Reporter,Global Trade Review,"Asset Recovery ,Compliance ,Fraud ,Insurance ,...",Journalist,United Kingdom (UK)
...,...,...,...,...,...,...,...
210,https://app.swapcard.com/event/offshorealert-l...,Abby Wilson,Investigative Journalism Student,"City, University of London","Corruption ,Fraud ,Intelligence ,Investigating...",Other,United States (US)
211,https://app.swapcard.com/event/offshorealert-l...,Colin Wilson,Director,KROLL,"Asset Recovery ,Compliance ,Corruption ,Crypto...",Director,British Virgin Islands\nCayman Islands
212,https://app.swapcard.com/event/offshorealert-l...,Dan Wise,Partner,Martin Kenney & Co (MKS),"Asset Recovery ,Compliance ,Corruption ,Crypto...",Attorney,British Virgin Islands
213,https://app.swapcard.com/event/offshorealert-l...,Iryna Wiseman,Director,Quantuma Advisory Limited,"Asset Recovery ,Compliance ,Corruption ,Crypto...",Accountant,British Virgin Islands\nCayman Islands\nUnited...


In [74]:
all_info.to_excel('Results/LondonEvent.xlsx', header=True, index=False)